# import library

In [1]:
import numpy as np
from scipy.spatial import distance
import ot
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.gaussian_process.kernels import RBF
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
import scipy as sp
import sys
sys.path.append('../')
from HOTDA import *
warnings.filterwarnings("ignore")
%matplotlib notebook
import time

# ----------------------------------USPS to MNIST-------------------------------------

In [2]:
data=sp.io.loadmat('./data/MNIST_vs_USPS.mat')
X_src=data['X_src']
Y_src=data['Y_src']
X_tar=data['X_tar']
Y_tar=data['Y_tar']

In [3]:
Y_src=Y_src.flatten()-1
Y_tar=Y_tar.flatten()-1
X_src=X_src.T
X_tar=X_tar.T

In [4]:
class Model_HOT_DA:
    def __init__(self):
        self.classifier = KNeighborsClassifier(n_neighbors=1)


        
    def fit(self,X,y,X_test,y_test):
        self.X = X 
        self.y=y
        self.X_test = X_test
        self.y_test = y_test
        self.clustering = SpectralClustering(n_clusters=10,eigen_solver='arpack', assign_labels='kmeans',affinity="nearest_neighbors",n_neighbors=7).fit(self.X_test)     #we can use wasserstein SpectralClustering instead of SpectralClustering
        S,a,mu,self.y_Transported_S=Source_target_processing(self.X,self.y)
        T,b,nu,yxyys=Source_target_processing(self.X_test,self.clustering.labels_)  
        HOT,W=Hot(S,a,mu,T,b,nu,0.2,1)
        self.X_Transported_S=Mapping(S,T,a,b,HOT,0.1)
        self.classifier = KNeighborsClassifier(n_neighbors=1)
        self.classifier.fit(self.X_Transported_S,self.y_Transported_S)
        
        
    def predict(self,X_test,y_test):
        self.y_test=y_test
        return self.classifier.score(self.X_test,y_test)

In [5]:
t=time.time()
mod=Model_HOT_DA()
mod.fit(X_src,Y_src,X_tar,Y_tar)
t2=time.time()
print("run time",t2-t)

run time 3.1443331241607666


In [6]:
mod.predict(X_tar,Y_tar)

0.7638888888888888